change it to not use the document when looking for the link quality, to see how much that is actually helping

also changing loss to be `gold - (max {other} + c)`

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from theano import *
from lasagne.layers import InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np
from helpers import SimpleMaxingLayer, SimpleAverageLayer
from wordvecs import WordVectors, EmbeddingLayer
import json
import re

theano.config.floatX = 'float32'
#theano.config.linker = 'cvm_nogc'
theano.config.openmp = True
theano.config.openmp_elemwise_minsize = 20000

Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
with open('/data/matthew/external-wiki2.json') as f:
    queries = json.load(f)['queries']

In [4]:
len(queries)

9915

In [5]:
sum([any([g['gold'] for g in v.values()]) for v in queries.values()])

8917

In [6]:
wordvectors = WordVectors(
    fname="/data/matthew/enwiki-20141208-pages-articles-multistream-links5-output1.bin",
    redir_fname='/data/matthew/enwiki-20141208-pages-articles-multistream-redirects5.json',
    negvectors=False,
    sentence_length=200,
)
wordvectors.add_unknown_words = False

In [7]:
# with open('/data/matthew/enwiki-20141208-pages-articles-multistream-redirects5.json') as f:
#     page_redirects = json.load(f)
page_redirects = wordvectors.redirects

In [8]:
len(wordvectors.vectors)

4613263

In [9]:
from wikireader import WikiRegexes, WikipediaReader

In [10]:
def PreProcessedQueries(wikipedia_dump_fname, wordvec=wordvectors, queries=queries, redirects=page_redirects):
    
    get_words = re.compile('[^a-zA-Z0-9 ]')
    get_link = re.compile('.*?\[(.*?)\].*?')
    
    queried_pages = set()
    for docs, q in queries.iteritems():
        wordvec.tokenize(docs, length=200)
        for sur, v in q.iteritems():
            wrds_sur = get_words.sub(' ', sur)
            wordvec.tokenize(wrds_sur)
            link_sur = get_link.match(sur).group(1)
            wordvec.tokenize(link_sur)
            for link in v['vals'].keys():
                wrds = get_words.sub(' ', link)
                wordvec.tokenize(wrds)
                tt = WikiRegexes.convertToTitle(link)
                wordvec.get_location(tt)
                queried_pages.add(tt)

    added_pages = set()
    for title in queried_pages:
        if title in redirects:
            #wordvec.tokenize(self.redirects[title])
            added_pages.add(redirects[title])
    queried_pages |= added_pages

    page_content = {}

#     class GetWikipediaWords(WikipediaReader, WikiRegexes):

#         def readPage(ss, title, content):
#             tt = ss.convertToTitle(title)
#             if tt in queried_pages:
#                 cnt = ss._wikiToText(content)
#                 page_content[tt] = wordvec.tokenize(cnt)

#     GetWikipediaWords(wikipedia_dump_fname).read()
    
    rr = redirects
    rq = queried_pages
    rc = page_content

    class PreProcessedQueriesCls(object):
        
        wordvecs = wordvec
        queries = queries
        redirects = rr
        queried_pages = rq
        page_content = rc
        
        
    return PreProcessedQueriesCls

In [13]:
basePreProcessedQueries = PreProcessedQueries('/data/matthew/enwiki-20141208-pages-articles-multistream.xml')

In [14]:
len(wordvectors.vectors)

4613263

In [20]:
class EntityVectorLinkExp(basePreProcessedQueries):
    
    batch_size = 1000 #20000
    num_training_items = 500000 #200000
    
    def __init__(self): #, wikipedia_dump_fname, wordvec=wordvectors, queries=queries, redirects=page_redirects):
        #self.wordvecs = wordvec
        #self.queries = queries
        self.sentence_length = self.wordvecs.sentence_length
        self.document_length = 100
        self.num_words_to_use_conv = 5
        #self.redirects = redirects
        #self.page_content = {}
        #self.wikipedia_dump_fname = wikipedia_dump_fname
        
        #self._process_queries()
        
        self._setup()
        
#     def _process_queries(self):
#         queried_pages = set()
#         for docs, q in self.queries.iteritems():
#             self.wordvecs.tokenize(docs)
#             for sur, v in q.iteritems():
#                 self.wordvecs.tokenize(sur)
#                 for link in v['vals'].keys():
#                     self.wordvecs.tokenize(link)
#                     tt = WikiRegexes.convertToTitle(link)
#                     #self.wordvecs.tokenize(tt)
#                     queried_pages.add(tt)

#         added_pages = set()
#         for title in queried_pages:
#             if title in self.redirects:
#                 #self.wordvecs.tokenize(self.redirects[title])
#                 added_pages.add(self.redirects[title])
#         queried_pages |= added_pages
        
#         self.queried_pages = queried_pages
                
#         class GetWikipediaWords(WikipediaReader, WikiRegexes):
            
#             def readPage(ss, title, content):
#                 tt = ss.convertToTitle(title)
#                 if tt in queried_pages:
#                     cnt = ss._wikiToText(content)
#                     self.page_content[tt] = self.wordvecs.tokenize(cnt)
        
#         GetWikipediaWords(self.wikipedia_dump_fname).read()
               
        
    def _setup(self):
        #self.x_document_input = T.imatrix('x_doc')
        
        #self.x_document_id = T.ivector('x_doc_id')
        self.x_surface_text_input = T.imatrix('x_surface_link')
        self.x_surface_context_input = T.imatrix('x_surface_cxt')  # TODO
        
        self.x_target_input = T.ivector('x_target')
        self.x_target_words = T.imatrix('x_target_words')
        self.x_matches_surface = T.ivector('x_match_surface')
        self.x_link_id = T.ivector('x_link_id')
        
        #self.y_score = T.vector('y')
        self.y_answer = T.ivector('y_ans')  # contains the location of the gold answer so we can compute the loss
        self.y_grouping = T.imatrix('y_grouping')
        
        
        self.embedding_W = theano.shared(self.wordvecs.get_numpy_matrix().astype(theano.config.floatX))
        
#         self.document_l = lasagne.layers.InputLayer(
#             (None,self.document_length), 
#             input_var=self.x_document_input
#         )
    
#         self.document_embedding_l = EmbeddingLayer(
#             self.document_l,
#             W=self.embedding_W,
#             add_word_params=False,
#         )
        
#         self.document_conv1_l = lasagne.layers.Conv2DLayer(
#             self.document_embedding_l,
#             num_filters=500,
#             filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
#             name='document_conv1',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )
        
#         self.document_max_l = lasagne.layers.Pool2DLayer(
#             self.document_conv1_l,
#             name='document_pool1',
#             pool_size=(self.document_length - self.num_words_to_use_conv, 1),
#             mode='sum',
#         )

#         self.document_dens1 = lasagne.layers.DenseLayer(
#             self.document_max_l,
#             num_units=250,
#             name='doucment_dens1',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )
        
#         self.document_drop1 = lasagne.layers.DropoutLayer(
#             self.document_dens1,
#             p=.25,
#         )
        
#         document_output_length = 200
        
#         self.document_dens2 = lasagne.layers.DenseLayer(
#             self.document_drop1,
#             num_units=225,
#             name='document_dens2',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )
        
#         self.document_drop2 = lasagne.layers.DropoutLayer(
#             self.document_dens2,
#             p=.25,
#         )
        
#         self.document_dens3 = lasagne.layers.DenseLayer(
#             self.document_drop2,
#             num_units=document_output_length,
#             name='document_dens3',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )
        
#         self.document_output = lasagne.layers.get_output(self.document_dens3)
        
        self.surface_context_l = lasagne.layers.InputLayer(
            (None, self.sentence_length),
            input_var=self.x_surface_context_input,
        )
        
        self.surface_context_embedding_l = EmbeddingLayer(
            self.surface_context_l,
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.surface_context_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_context_embedding_l,
            num_filters=300,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_cxt_conv1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
#         self.surface_context_avg1_l = SimpleAverageLayer(
#             [self.surface_context_conv1_l, self.surface_context_l],
#             #name='surface_context_avg'
#         )
        
        self.surface_context_pool1_l = lasagne.layers.Pool2DLayer(
            self.surface_context_conv1_l,
            name='surface_cxt_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )
                
        self.surface_input_l = lasagne.layers.InputLayer(
            (None, self.sentence_length), 
            input_var=self.x_surface_text_input
        )
        
        self.surface_embedding_l = EmbeddingLayer(
            self.surface_input_l,
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.surface_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_embedding_l,
            num_filters=300,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_conv1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
#         self.surface_avg1_l = SimpleAverageLayer(
#             [self.surface_conv1_l, self.surface_input_l],
#             #name='surface_avg'
#         )
        
        self.surface_pool1_l = lasagne.layers.Pool2DLayer(
            self.surface_conv1_l,
            name='surface_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )
        
        self.surface_merged_l = lasagne.layers.ConcatLayer(
            [self.surface_context_pool1_l, self.surface_pool1_l]
        )
        
        self.surface_dens1 = lasagne.layers.DenseLayer(
            self.surface_merged_l,
            name='surface_dens1',
            num_units=250,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
#         self.surface_drop1 = lasagne.layers.DropoutLayer(
#             self.surface_dens1,
#             p=.25,
#         )
        
#         self.surface_dens2 = lasagne.layers.DenseLayer(
#             self.surface_drop1,
#             name='surface_dens2',
#             num_units=200,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )
        
#         self.document_aligned_l = InputLayer(
#             (None, document_output_length),
#             input_var=self.document_output[self.x_document_id,:]
#         )
        
        ##############################################
        ## changed to not use the documented
        
#         self.source_l = lasagne.layers.ConcatLayer(
#             [self.document_aligned_l, self.surface_dens1]
#         )
        
        self.source_dens1 = lasagne.layers.DenseLayer(
            self.surface_dens1,   # CHANGED
            num_units=300,
            name='source_dens1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.source_drop1 = lasagne.layers.DropoutLayer(
            self.source_dens1,
            p=.25,
        )
        
        self.source_dens12 = lasagne.layers.DenseLayer(
            self.source_drop1,
            num_units=250,
            name='source_dens12',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.source_drop12 = lasagne.layers.DropoutLayer(
            self.source_dens12,
            p=.25,
        )
        
        compared_vector_size = self.wordvecs.vector_size #+ 2 # extra space for if it matches the surface text
        
        self.source_dens2 = lasagne.layers.DenseLayer(
            self.source_drop12,
            num_units=compared_vector_size,  # this is the same size as the learned wikipedia vectors
            name='source_dens2',
            nonlinearity=lasagne.nonlinearities.linear,
        )
        
        self.source_out = lasagne.layers.get_output(self.source_dens2)
        
        matched_surface_reshaped = self.x_matches_surface.reshape(
            (self.x_matches_surface.shape[0], 1, 1, 1)).astype(theano.config.floatX)
                
        self.target_input_l = lasagne.layers.InputLayer(
            (None,),
            input_var=self.x_target_input
        )
        
        self.target_matched_surface_input_l = lasagne.layers.InputLayer(
            (None,1,1,1),
            input_var=matched_surface_reshaped,
        )
        
        self.target_embedding_l = EmbeddingLayer(
            lasagne.layers.reshape(self.target_input_l, ([0], 1)),
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.target_combined_feats_l = lasagne.layers.ConcatLayer(
            [self.target_embedding_l, self.target_matched_surface_input_l],
            axis=3
        )
        
        self.target_words_input_l = lasagne.layers.InputLayer(
            (None,self.sentence_length),
            input_var=self.x_target_words,
        )
        
        self.target_words_embedding_l = EmbeddingLayer(
            self.target_words_input_l,
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.target_words_conv1_l = lasagne.layers.Conv2DLayer(
            self.target_words_embedding_l,
            name='target_wrds_conv1',
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            num_filters=350,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.target_words_pool1_l = lasagne.layers.Pool2DLayer(
            self.target_words_conv1_l,
            name='target_wrds_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )
        
        self.target_merge_l = lasagne.layers.ConcatLayer(
            [lasagne.layers.reshape(self.target_words_pool1_l, ([0], [1])),
             lasagne.layers.reshape(self.target_embedding_l, ([0], [3]))]
        )
        
        self.target_dens1 = lasagne.layers.DenseLayer(
            self.target_merge_l,
            name='target_wrds_dens1',
            num_units=400,
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.target_drop1 = lasagne.layers.DropoutLayer(
            self.target_dens1,
            p=.25,
        )
        
        self.target_dens2 = lasagne.layers.DenseLayer(
            self.target_drop1,
            name='target_wrds_dens1',
            num_units=compared_vector_size,
            nonlinearity=lasagne.nonlinearities.linear,
        )
        
        self.target_simple = lasagne.layers.DenseLayer(
            self.target_combined_feats_l,
            name='target_simple1',
            num_units=compared_vector_size,
            nonlinearity=lasagne.nonlinearities.linear,
        )
        
#         self.target_dens1 = lasagne.layers.DenseLayer(
#             self.target_conv1_l,
#             name='target_dens1',
#             num_units=300,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )
        
#         self.target_drop1 = lasagne.layers.DropoutLayer(
#             self.target_dens1,
#             p=.25,
#         )
        
#         self.target_dens2 = lasagne.layers.DenseLayer(
#             self.target_drop1,
#             name='target_dens2',
#             num_units=300,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )
        
        
    
        #self.target_out = lasagne.layers.get_output(self.target_embedding_l)
        
        
#         self.target_out = T.concatenate(
#             [self.embedding_W[self.x_target_input],
#              matched_surface_reshaped,
#             1-matched_surface_reshaped],
#              axis=1)
        
        
        #self.target_out = self.embedding_W[self.x_target_input]
        #self.target_out = lasagne.layers.get_output(self.target_dens2)
        
        self.target_out = lasagne.layers.get_output(self.target_simple)
        
        # compute the cosine distance between the two layers
        self.source_aligned_l = self.source_out[self.x_link_id, :]
        
        # this uses scan internally, which means that it comes back into python code to run the loop.....fml
        self.dotted_vectors =  T.batched_dot(self.target_out, self.source_aligned_l)
        # diag also does not support a C version.........
        #self.dotted_vectors = T.dot(self.target_out, self.source_aligned_l.T).diagonal()
        
        def augNorm(v):
            return T.maximum(T.basic.pow(T.basic.pow(T.basic.abs_(v), 2).sum(axis=1) + .001, .5), .001)
    
        self.res_l = self.dotted_vectors / (augNorm(self.target_out) * augNorm(self.source_aligned_l) + .001)
#         self.res_l = self.dotted_vectors / ((self.target_out.norm(1, axis=1) + .001) * 
#                                             (self.source_aligned_l.norm(1, axis=1) + .001))
        
        self.res_cap = T.clip((T.tanh(self.res_l) + 1) / 2, .001, .999)
        
        #self.golds = self.res_cap[self.y_answer]
                
#         def maxOverRange(indx):
#             #return T.max(self.res_cap[T.arange(indx[0],indx[1])]) - self.res_cap[indx[2]]
#             #return -( self.res_l[indx[2]] - T.log(T.exp(self.res_l[T.arange(indx[0],indx[1])]).sum()) )
#             return -( self.res_l[indx[2]] - self.res_l[indx[0]])
        
#         # build a tensor to make a matrix with one set on each dimention
#         self.grouped, grouped_update = theano.scan(maxOverRange, sequences=self.y_grouping)
        
        def setSubSelector(indx, outputs):
            r = T.set_subtensor(outputs[T.arange(indx[0], indx[1]), indx[3]], 1)
            return T.set_subtensor(r[indx[2], indx[3]], 0)  # set the gold item to zero
            #return T.set_subtensor(outputs[T.arange(indx[0], indx[1]), indx[3]], 1)
        
        num_target_samples = self.res_l.shape[0]
        
        select_seq = T.concatenate([
            self.y_grouping,
            T.arange(self.y_grouping.shape[0]).reshape((self.y_grouping.shape[0], 1))
        ], axis=1)
        
        self.selecting_matrix, _ = theano.scan(
            setSubSelector,
            outputs_info=T.zeros((num_target_samples, num_target_samples)),
            #n_steps=self.y_grouping.shape[0]
            sequences=select_seq,
        )
        
        self.groupped_elems = T.dot(self.selecting_matrix[-1], T.diag(self.res_l))
        #self.groupped_res = T.log(self.groupped_elems.sum(axis=0)[T.arange(self.y_grouping.shape[0])])
        self.groupped_res = self.groupped_elems.max(axis=0)[T.arange(self.y_grouping.shape[0])]
        self.loss_vec = T.max(self.groupped_res + 1 - self.res_l[self.y_grouping[:,2]], 0)
        
        self.all_params = (
            #lasagne.layers.get_all_params(self.target_dens2) +
            # TODO: add params for the target stuff, 
            lasagne.layers.get_all_params(self.target_simple) +
            lasagne.layers.get_all_params(self.source_dens2)
            #lasagne.layers.get_all_params(self.document_dens2)
        )
        
        # weight the positive samples more since there are fewer of them,
        # freaking hack
        #self.loss_vec = -(10 * self.y_score * T.log(self.res_cap) + (1.0 - self.y_score) * T.log(1.0 - self.res_cap))
        
        #self.loss_vec = T.nnet.binary_crossentropy(self.res_cap, self.y_score)
        
        #self.loss_vec = T.exp(T.max(self.res_cap - self.res_cap[self.y_answer] + .1, 0)) - 1  # TODO: maybe have some squared term here or something?
        
        # this one works reasonably well
        #self.loss_vec = - T.log((T.clip(self.res_cap[self.y_answer] - self.res_cap, -1.0, 0.4) + 1.0) / 1.5)
        
        #self.loss_vec = self.grouped
        
        #self.loss_vec = - T.log((T.clip(self.res_l[self.y_answer] - self.res_l, -40.0, 10.0) + 40.0) / 51.0)
        #self.loss_vec = T.max(self.res_l[self.y_answer] - self.res_l + .1, 0)
        
        self.updates = lasagne.updates.adadelta(self.loss_vec.mean(), self.all_params)
        
        self.func_inputs = [
            #self.x_document_input,
            self.x_surface_text_input, self.x_surface_context_input, #self.x_document_id,
            self.x_target_input, self.x_matches_surface, self.x_link_id, 
            self.y_answer, self.y_grouping
        ]  # self.x_target_words,
        
        self.train_func = theano.function(
            self.func_inputs,
            [self.res_cap, self.loss_vec.sum(), self.loss_vec],
            updates=self.updates,
            on_unused_input='ignore',
        )
        
        self.test_func = theano.function(
            self.func_inputs,
            [self.res_cap, self.loss_vec.sum(), self.loss_vec],
            on_unused_input='ignore',
        )
        
    def reset_accums(self):
        self.current_documents = []
        self.current_surface_context = []
        self.current_surface_link = []
        self.current_link_id = []
        self.current_target_input = []
        self.current_target_words = []
        self.current_target_matches_surface = []
        self.current_target_id = []
        self.current_target_goal = []
        self.current_learning_groups = []
        self.learning_targets = []
        
    def compute_batch(self, isTraining=True):
        if isTraining:
            func = self.train_func
        else:
            func = self.test_func
        self.reset_accums()
        self.total_links = 0
        self.total_loss = 0.0
        
        get_words = re.compile('[^a-zA-Z0-9 ]')
        get_link = re.compile('.*?\[(.*?)\].*?')
        
        for doc, queries in self.queries.iteritems():
            # skip the testing documents while training and vice versa
            if queries.values()[0]['training'] != isTraining:
                continue
            docid = len(self.current_documents)
            self.current_documents.append(self.wordvecs.tokenize(doc, length=self.document_length))
            for surtxt, targets in queries.iteritems():
                self.current_link_id.append(docid)
                surid = len(self.current_surface_link)
                self.current_surface_context.append(self.wordvecs.tokenize(get_words.sub(' ' , surtxt)))
                surlink = get_link.match(surtxt).group(1)
                self.current_surface_link.append(self.wordvecs.tokenize(surlink))
                surmatch = surlink.lower()
                #target_page_input = []
                target_words_input = []
                target_matches_surface = []
                target_inputs = []
                target_learings = []
                target_gold_loc = -1
                target_group_start = len(self.current_target_input)
                for target in targets['vals'].keys():
                    # skip the items that we don't know the gold for
                    if not targets['gold'] and isTraining:
                        continue
                    isGold = target == targets['gold']
                    #cnt = self.page_content.get(WikiRegexes.convertToTitle(target))
                    cnt = self.wordvecs.get_location(WikiRegexes.convertToTitle(target))
                    if cnt is None:
                        # were not able to find this wikipedia document
                        # so just ignore tihs result since trying to train on it will cause
                        # issues
                        continue
                    if isGold:
                        target_gold_loc = len(target_inputs)
                    #target_page_input.append(cnt)
                    target_words_input.append(self.wordvecs.tokenize(get_words.sub(' ', target)))
                    target_inputs.append(cnt)  # page_content already tokenized
                    target_matches_surface.append(int(surmatch == target.lower()))
                    target_learings.append((targets, target))
                if target_gold_loc is not None or not isTraining:  # if we can't get the gold item
                    # contain the index of the gold item for these items, so it can be less then it
                    gold_loc = (len(self.current_target_goal) + target_gold_loc)
                    self.current_target_goal += [gold_loc] * len(target_inputs)
                    self.current_target_input += target_inputs
                    self.current_target_id += [surid] * len(target_inputs)
                    self.current_target_words += target_words_input   # TODO: add
                    self.current_target_matches_surface += target_matches_surface
                    target_group_end = len(self.current_target_input)
                    self.current_learning_groups.append(
                        [target_group_start, target_group_end,
                         gold_loc])
                
                #self.current_target_goal.append(isGold)
                self.learning_targets += target_learings
            if len(self.current_target_id) > self.batch_size:
                #return
                self.run_batch(func)
                if self.total_links > self.num_training_items:
                    return self.total_loss / self.total_links
        
        if len(self.current_target_id) > 0:
            self.run_batch(func)
            
        return self.total_loss / self.total_links
        
    def run_batch(self, func):
        res_vec, loss_sum, loss_vec = func(
            #self.current_documents,
            self.current_surface_link, self.current_surface_context, #self.current_link_id,
            self.current_target_input, self.current_target_matches_surface, self.current_target_id, 
            self.current_target_goal, self.current_learning_groups,  
            # self.current_target_words,
        )
        self.check_params()
        self.total_links += len(self.current_target_id)
        self.total_loss += loss_sum
        for i in xrange(len(res_vec)):
            # save the results from this pass
            l = self.learning_targets[i]
            l[0]['vals'][ l[1] ] = res_vec[i]
        self.reset_accums()
        
    def check_params(self):
        if any([np.isnan(v.get_value(borrow=True)).any() for v in self.all_params]):
            raise RuntimeError('nan in some of the parameters')
        

        
queries_exp = EntityVectorLinkExp() 

In [21]:
queries_exp.check_params()

In [96]:
len(queries_exp.current_documents),set(queries_exp.current_link_id)

(5, {0, 1, 2, 3, 4})

In [98]:
len(queries_exp.current_surface_link),len(set(queries_exp.current_target_id))

(87, 87)

In [99]:
len(queries_exp.current_learning_groups)

87

In [95]:
len(queries_exp.current_target_id)

1184

In [100]:
queries_exp.total_links

0

In [22]:
queries_exp.num_training_items = 50000

In [18]:
%time print queries_exp.compute_batch(False)

0.190234110474
CPU times: user 1h 31min 40s, sys: 25.4 s, total: 1h 32min 5s
Wall time: 12min 10s


In [ ]:
%time print queries_exp.compute_batch()

In [25]:
evalCurrentState(False, 500000)

(28422, 0.0)


(28422, 0.0)

In [26]:
evalCurrentState(True, 500000)

(28325, 0.0)


(28325, 0.0)

In [ ]:
exp_results = []

for i in xrange(5):
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)
exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('training state', evalCurrentState(True)))
exp_results.append(('testing state', evalCurrentState(False)))

In [25]:
exp_results

[(0, 0.0012442890062829277),
 (1, 0.0012170055321185696),
 (2, 0.0011066949465393097),
 (3, 0.0010886187919973039),
 (4, 0.001062282717236833),
 ('testing run', 0.0011542985617437999)]

In [27]:
evalCurrentState(True)

(2846, 0.07273366127898806)


(2846, 0.07273366127898806)

In [28]:
evalCurrentState(False)

ZeroDivisionError: float division by zero

In [111]:
evalCurrentState(True, 50000000)

(517581, 0.1446556191204855)


(517581, 0.1446556191204855)

In [114]:
evalCurrentState(False, 500000)

(28420, 0.6482758620689655)


(28420, 0.6482758620689655)

In [ ]:
queries_exp.num_training_items = 1000000

exp_results.append('increase training item size')
exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('testing state', evalCurrentState(False, 10000000)))

for i in xrange(5):
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)
exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('training state', evalCurrentState(True, 10000000)))
exp_results.append(('testing state', evalCurrentState(False, 10000000)))

In [116]:
exp_results

[(0, 0.13266118701557572),
 (1, 0.12355173346257406),
 (2, 0.11546770283164702),
 (3, 0.1092563206515393),
 (4, 0.10597121073958943),
 ('testing run', 0.10553999777445534),
 ('training state', (2837, 0.6669016566795911)),
 ('testing state', (2911, 0.6413603572655445)),
 'increase training item size',
 ('testing run', 0.10553945666484769),
 ('testing state', (93705, 0.6441491916119737)),
 (0, 0.10374660023013943),
 (1, 0.10251044899899585),
 (2, 0.10196588812502484),
 (3, 0.10156279893654875),
 (4, 0.10114781341023653),
 ('testing run', 0.10223237171087719),
 ('training state', (517581, 0.29967483350432106)),
 ('testing state', (93705, 0.6602529214022731))]

In [76]:
exp_results

[(0, 0.17593555299493149),
 (1, 0.15775220255413391),
 (2, 0.14848182698742496),
 (3, 0.14236914357040895),
 (4, 0.13783007839308653),
 (5, 0.13423812280731193),
 (6, 0.13121966899002982),
 (7, 0.12862972168658454),
 (8, 0.12631712098052536),
 (9, 0.12426717171407249),
 ('testing run', 0.1413455562823264),
 ('training state', (2837, 0.4532957349312654)),
 ('testing state', (2911, 0.4002061147372037))]

In [ ]:
for i in xrange(20):
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)

exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('training state', evalCurrentState(True)))
exp_results.append(('testing state', evalCurrentState(False)))

In [ ]:
for i in xrange(40):
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)

exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('training state', evalCurrentState(True)))
exp_results.append(('testing state', evalCurrentState(False)))

In [79]:
exp_results

[(0, 0.17593555299493149),
 (1, 0.15775220255413391),
 (2, 0.14848182698742496),
 (3, 0.14236914357040895),
 (4, 0.13783007839308653),
 (5, 0.13423812280731193),
 (6, 0.13121966899002982),
 (7, 0.12862972168658454),
 (8, 0.12631712098052536),
 (9, 0.12426717171407249),
 ('testing run', 0.1413455562823264),
 ('training state', (2837, 0.4532957349312654)),
 ('testing state', (2911, 0.4002061147372037)),
 (0, 0.12236985757816354),
 (1, 0.12059972496635762),
 (2, 0.11895977317063225),
 (3, 0.1174358839507175),
 (4, 0.11597574485376827),
 (5, 0.11460117983657973),
 (6, 0.11325147423622688),
 (7, 0.111944676801859),
 (8, 0.11068471110399238),
 (9, 0.10949150958354345),
 (10, 0.10830747762660752),
 (11, 0.10713663977902753),
 (12, 0.10603463418394098),
 (13, 0.10496975296489068),
 (14, 0.10389930968826112),
 (15, 0.10287206584124167),
 (16, 0.10198909359236984),
 (17, 0.10101071914622954),
 (18, 0.10005361757370523),
 (19, 0.099186495055756704),
 ('testing run', 0.1268706949775481),
 ('traini

In [82]:
queries_exp.num_training_items = 500000
exp_results.append('increse size to 500,000')

exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('training state', evalCurrentState(True, 500000)))
exp_results.append(('testing state', evalCurrentState(False, 500000)))

for i in xrange(20):
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)
    
exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('training state', evalCurrentState(True, 500000)))
exp_results.append(('testing state', evalCurrentState(False, 500000)))

(28207, 0.21292586946502642)
(28420, 0.0)


RuntimeError: nan in some of the parameters

In [92]:
[(v, np.isnan(v.get_value(borrow=True)).all()) for v in queries_exp.all_params]

[(target_simple1.W, True),
 (target_simple1.b, True),
 (surface_cxt_conv1.W, True),
 (surface_cxt_conv1.b, True),
 (surface_conv1.W, True),
 (surface_conv1.b, True),
 (surface_dens1.W, True),
 (surface_dens1.b, True),
 (source_dens1.W, True),
 (source_dens1.b, True),
 (source_dens12.W, True),
 (source_dens12.b, True),
 (source_dens2.W, True),
 (source_dens2.b, True),
 (document_conv1.W, True),
 (document_conv1.b, True),
 (doucment_dens1.W, True),
 (doucment_dens1.b, True),
 (document_dens2.W, True),
 (document_dens2.b, True)]

In [87]:
queries_exp.check_params()

RuntimeError: nan in some of the parameters

In [249]:
queries_exp.compute_batch(False)

ZeroDivisionError: float division by zero

In [33]:
def qwerqwer():
    tt = 0
    for qu in queries_exp.queries.values():
        if qu.values()[0]['training'] == True:
            pass
        else:
            tt += 1
#         for en in qu.values():
#             for e in en:
#                 if en['training'] is False:
#                     tt += 1
    return tt
qwerqwer()

998

In [26]:
def evalCurrentState(trainingData=True, numSamples=50000):
    all_measured = 0
    all_correct = 0
    all_trained = 0
    for qu in queries.values():
        for en in qu.values():
            if en['training'] != trainingData:
                continue
            for e in en:
                if en['gold']:
                    if all_trained > numSamples:
                        break
                    all_measured += 1
                    all_trained += len(en['vals'].values())
                    m = max(en['vals'].values())
                    if en['vals'][en['gold']] == m and m != 0:
                        all_correct += 1
           
    r = all_measured, float(all_correct) / all_measured
    print r
    return r

In [63]:
evalCurrentState(False, 10000000)

(91257, 0.0012492192379762648)


(91257, 0.0012492192379762648)

In [23]:
import random
def augmentTrainingData():
    for quk in queries.keys():
        qu = queries[quk]
        for enk in qu.keys():
            en = qu[enk]
            if not en['gold']:
                del qu[enk]
        if not qu:
            del queries[quk]
    for qu in queries.values():
        training = random.random() > .15
        for en in qu.values():
            en['training'] = training
augmentTrainingData()

In [234]:
queries.values()[29].values()[0]['training']

True

In [230]:
queries.values()[30]

{u' [Kidnapping] charges were dropped again Vivian .': {u'gold': u'',
  u'training': False,
  u'vals': {u'-NIL-': 0,
   u'Chibok schoolgirls kidnapping': 0,
   u'Kidnapping': 0}},
 u", Vivian took Nicky away during Carly 's wedding to [Bo Brady] .": {u'gold': u'',
  u'training': False,
  u'vals': {u'-NIL-': 0,
   u'Bo Brady': 0,
   u'Brady': 0,
   u'Brady Township, Butler County, Pennsylvania': 0,
   u'Brady Township, Clarion County, Pennsylvania': 0,
   u'Brady Township, Lycoming County, Pennsylvania': 0,
   u'Brady family': 0,
   u'Brady, Montana': 0,
   u'Brady, Nebraska': 0,
   u'Brady, Texas': 0,
   u'Darren Brady': 0,
   u'Garry Brady': 0,
   u'Greg Brady (broadcaster)': 0,
   u'Jon Brady': 0,
   u'Liam Brady': 0,
   u'Mathew Brady': 0,
   u'Robbie Brady': 0,
   u'Shannon Brady': 0,
   u'Tom Brady': 0,
   u'Tom Brady (rugby union)': 0}},
 u'After John went to [Lugano] with Danielle , she followed and discovered that Danielle was': {u'gold': u'',
  u'training': False,
  u'vals': {

In [142]:
queries_exp.compute_batch()

In [88]:
gg_func = theano.function(
    queries_exp.func_inputs,
    [queries_exp.loss_vec, queries_exp.source_out, queries_exp.target_out] + T.grad(queries_exp.loss_vec.mean(), queries_exp.all_params),
    updates=queries_exp.updates,
    on_unused_input='ignore',
)

In [91]:
res1 = gg_func(
    queries_exp.current_documents,
    queries_exp.current_surface_link, queries_exp.current_surface_context, queries_exp.current_link_id,
    queries_exp.current_target_input, queries_exp.current_target_matches_surface, queries_exp.current_target_id, queries_exp.current_target_goal
)

In [92]:
res1[0].max(), res1[0].mean(), res1[0].min()

(0.24814822, 0.24238527, 0.23680143)

In [81]:
np.dot(res1[1][0], res1[2][0:10,:].T)

array([  76.69731903,  172.44116211,  139.30476379, -182.91343689,
        172.44116211,  172.44116211,  -13.79522324,   97.6740036 ,
         11.84405231,   36.98944092], dtype=float32)

In [83]:
res1[0][0:10]

array([ 0.24216416,  0.24294616,  0.24274334,  0.23833458,  0.24294616,
        0.24294616,  0.24024701,  0.24193516,  0.24127102,  0.24146625], dtype=float32)

In [71]:
res1[3:][0].std()

9.7740622e-06

In [53]:
queries_exp.all_params

[surface_cxt_conv1.W,
 surface_cxt_conv1.b,
 surface_conv1.W,
 surface_conv1.b,
 surface_dens1.W,
 surface_dens1.b,
 source_dens1.W,
 source_dens1.b,
 source_dens12.W,
 source_dens12.b,
 source_dens2.W,
 source_dens2.b,
 document_conv1.W,
 document_conv1.b,
 doucment_dens1.W,
 doucment_dens1.b,
 document_dens2.W,
 document_dens2.b]

In [197]:
res1[0].mean()

0.24253285

In [198]:
np.dot(res1[2][0,:],res1[1][0:15,:].T)

array([-10.04426765,  -8.85146523,  -6.5927825 ,  -5.64352655,
        -6.2810235 , -10.34654236,  -6.09899616,   1.8380394 ,
        -3.23892975,  -6.04222679,  -8.91859531,   1.09592962,
        -5.27822304, -12.33982086,  -5.90433264], dtype=float32)

In [178]:
res1[2][0,:].min()

-5.4103508

In [168]:
res1[1][0,:]

-0.82824624

In [ ]:
np.isnan(gg_res[0]).any()

In [ ]:
gg_res[2].mean()

In [ ]:
gg_res[0].mean()

In [ ]:
[(v, np.isnan(v.get_value(borrow=True)).all()) for v in queries_exp.all_params]

In [ ]:
queries_exp.total_links

In [ ]:
theano.printing.pydotprint(T.grad(queries_exp.loss_vec.mean(), queries_exp.all_params))

In [ ]:
gg_func = theano.function(
            [queries_exp.x_document_input,
             queries_exp.x_surface_text_input, queries_exp.x_document_id,
             queries_exp.x_target_input, queries_exp.x_link_id, queries_exp.y_score],
            T.grad(queries_exp.loss_vec.mean(), lasagne.layers.get_all_params(queries_exp.target_dens2)),
#           T.grad(queries_exp.loss_vec.mean(), queries_exp.all_params),
    #             [queries_exp.target_out, queries_exp.source_aligned_l, 
#              T.dot(queries_exp.target_out, queries_exp.source_aligned_l.T).diagonal(),
#              queries_exp.target_out.norm(2, axis=1) * queries_exp.source_aligned_l.norm(2, axis=1),
#              T.batched_dot(queries_exp.target_out, queries_exp.source_aligned_l),
#              lasagne.layers.get_output(queries_exp.target_dens2),
#              queries_exp.target_out.norm(2, axis=1),
#              #T.grad(queries_exp.loss_vec.mean(), queries_exp.all_params)
#             ],
        #[queries_exp.res_l, queries_exp.loss_vec.sum(), queries_exp.loss_vec],
    on_unused_input='ignore',
    mode='DebugMode'
        )

In [19]:
gg_func = theano.function(
            [queries_exp.x_document_input,
             queries_exp.x_surface_text_input, queries_exp.x_document_id,
             queries_exp.x_target_input, queries_exp.x_link_id, queries_exp.y_answer],
            #[self.res_cap, self.loss_vec.sum(), self.loss_vec],
            [queries_exp.res_cap],
        mode='DebugMode',
        on_unused_input='ignore',
        )

In [20]:
try:
    gg_grad_res = gg_func(
        queries_exp.current_documents,
        queries_exp.current_surface_text, queries_exp.current_link_id,
        queries_exp.current_target_input, queries_exp.current_target_id, queries_exp.current_target_goal
    )
except Exception as e:
    eeee = e

In [21]:
eeee

ValueError('Bad input argument to theano function with name "<ipython-input-19-c6e818cc55a5>:8"  at index 4(0-based)',
           'setting an array element with a sequence.')

In [ ]:
[np.isnan(v).any() for v in gg_grad_res]

In [ ]:
eeee

In [ ]:
len(queries_exp.current_target_input)

In [ ]:
gg_res = gg_func(
    queries_exp.current_documents,
    queries_exp.current_surface_text, queries_exp.current_link_id,
    queries_exp.current_target_input, queries_exp.current_target_id, queries_exp.current_target_goal
)

In [ ]:
len(queries_exp.current_target_input)

In [ ]:
np.isnan(gg_res[5]).any()

In [ ]:
gg_res[2]

In [ ]:
gg_res[0].shape, gg_res[1].shape, gg_res[2].shape, gg_res[3].shape, gg_res[4].shape

In [ ]:
np.inner(gg_res[0], gg_res[1]).shape

In [ ]:
gg_res[0] * gg_res[1]

In [ ]:
aa = np.dot(gg_res[0], gg_res[1].T)

In [ ]:
aa.diagonal().shape

In [ ]:
gg_res[2]

In [ ]:
len(queries_exp.queried_pages)

In [ ]:
len(queries_exp.page_content)

In [ ]:
len(queries_exp.current_surface_text)

In [ ]:
len(queries_exp.current_link_id)

In [ ]:
len(queries_exp.current_target_input)

In [ ]:
exp_results = []

for i in xrange(5):
    exp_results.append((i, queries_exp.compute_batch()))

In [ ]:
exp_results

In [ ]:
queries.values()

In [ ]:
queries_exp.total_links

In [ ]:
queries_exp.total_loss / queries_exp.total_links

In [ ]:
queries_exp.total_loss

In [ ]:
len(queries_exp.current_target_id)

In [ ]:
queries_exp.compute_batch()

In [ ]:
%time queries_exp.run_batch(queries_exp.test_func)